In [3]:
import torch

a = torch.ones(3)
b = torch.ones(3)
torch.stack((a, b), dim=1).shape


torch.Size([3, 2])

In [5]:
import pickle

# 定义新文件路径
goodreads_children_path = 'Dataset/goodreads_children/processed/children.pkl'

# 加载 goodreads_children.pkl 文件
with open(goodreads_children_path, 'rb') as f:
    goodreads_children_data = pickle.load(f)

# 打印加载的数据
print(goodreads_children_data)

Data(edge_index=[2, 734640], text_nodes=[216613], text_edges=[734640], text_node_labels=[216613], edge_score=[734640])


In [7]:
# train: use a whole graph and train on specific edges 

In [3]:
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import LinkNeighborLoader

data = Planetoid("Cora", name='Cora')[0]

loader = LinkNeighborLoader(
    data,
    # Sample 30 neighbors for each node for 2 iterations
    num_neighbors=[30] * 2,
    # Use a batch size of 128 for sampling training nodes
    batch_size=128,
    edge_label_index=data.edge_index,
)

sampled_data = next(iter(loader))
print(sampled_data)


Processing...
Done!


Data(x=[1364, 1433], edge_index=[2, 3101], y=[1364], train_mask=[1364], val_mask=[1364], test_mask=[1364], n_id=[1364], e_id=[3101], num_sampled_nodes=[3], num_sampled_edges=[2], input_id=[128], edge_label_index=[2, 128])


In [ ]:
from torch_geometric.loader import NeighborLoader

loader = NeighborLoader(
    data,
    num_neighbors=[30] * 2,
    batch_size=128,
    edge_label_index=data.edge_index,
)

sampled_data = next(iter(loader))
print(sampled_data)

In [1]:
from torch_geometric.datasets import Reddit
dataset = Reddit("Reddit")

data = dataset[0]


In [2]:
data

Data(x=[232965, 602], edge_index=[2, 114615892], y=[232965], train_mask=[232965], val_mask=[232965], test_mask=[232965])

In [3]:
from torch_geometric.loader.utils import get_input_nodes
input_type, input_nodes, input_id = get_input_nodes(data, data.train_mask, None)

input_type, input_nodes, input_id

(None,
 tensor([     1,      3,      4,  ..., 232961, 232962, 232963]),
 tensor([     1,      3,      4,  ..., 232961, 232962, 232963]))

In [4]:
from torch_geometric.sampler import NodeSamplerInput
sample_data = NodeSamplerInput(
            input_id=input_id,
            node=input_nodes,
            time=None,
            input_type=input_type,
        )

In [2]:
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import LinkNeighborLoader
import torch

data = Planetoid("Cora", name='Cora')[0]

loader = LinkNeighborLoader(
    data,
    # Sample 30 neighbors for each node for 2 iterations
    num_neighbors=[30] * 2,
    # Use a batch size of 128 for sampling training nodes
    batch_size=1,
    edge_label_index=data.edge_index[:, 0:2],
)

sampled_data = next(iter(loader))
print(sampled_data)

Data(x=[38, 1433], edge_index=[2, 45], y=[38], train_mask=[38], val_mask=[38], test_mask=[38], n_id=[38], e_id=[45], num_sampled_nodes=[3], num_sampled_edges=[2], input_id=[1], edge_label_index=[2, 1])
